In [78]:
import os
import re
import pandas as pd

In [7]:
# get all subdirectories in data folder
data_path = r'../data'
dataset_labels = []
for dir in os.scandir(data_path):
    dataset_labels.append(dir.name)

In [42]:
# get all log files for each data set
dataset_logs = []
log_pattern = r'[A-Za-z0-9]*.log'
for i, dir_list in enumerate([os.listdir(data_path + '/' + dir) for dir in dataset_labels]):
    specific_log_title = []
    specific_log = [] 
    for file in dir_list:
        if re.search(log_pattern, file):
            specific_log_title.append(file)
            f = open(data_path + '/' + dataset_labels[i] + '/' + file)
            specific_log.append(f.read())
            f.close()
    dataset_logs.append((specific_log_title,specific_log))



In [71]:
log = dataset_logs[0][1]
split_logs = str(log).split(r'end of world_state')
split_logs = split_logs[:-1]
len(split_logs)

20

In [73]:
for i, log in enumerate(split_logs):
    split_logs[i] = log.split('end of events')[0]
    
flight_envelopes = [] # contains all timestep data
events = [] # contains the events at the end of each log file
for i, log in enumerate(split_logs):
    flight_envelopes.append(log.split('events')[0])
    events.append(log.split('events')[1])

In [92]:

df_arrays = []
for i in flight_envelopes[0].split(r'\n'):
    df_arrays.append(i.split(','))
df = pd.DataFrame(df_arrays)


In [119]:
df.columns = [re.search(r'([A-Za-z0-9(/)]*):', s).group(0) for s in df.iloc[0,:]]

In [142]:
sub_pattern = r'["[\' \sA-Za-z0-9(/)]*:'
re.sub(sub_pattern, '', df.iloc[0,0])

' 0.00'

In [140]:
df.iloc[0,2]

' speed(m/s): 240.83'

In [159]:
df = df[df['x:'].notnull()]

In [164]:
df

,Player:,MSLAlt:,speed(m/s):,heading:,x:,y:,z:,pitch:,bank:,yaw:
0,['Player: t: 0.00,MSLAlt: 1279.81,speed(m/s): 240.83,heading: 5.50,x: -302927.78,y: 1279.81,z: 678108.13,pitch: 1.74,bank: -0.00,yaw: 314.88 Emitters.Mode: 0
1,Player: t: 0.02,MSLAlt: 1279.81,speed(m/s): 240.83,heading: 5.50,x: -302924.88,y: 1279.81,z: 678105.21,pitch: 1.74,bank: 0.00,yaw: 314.88 Emitters.Mode: 0
2,Player: t: 0.04,MSLAlt: 1279.81,speed(m/s): 240.83,heading: 5.50,x: -302921.44,y: 1279.81,z: 678101.75,pitch: 1.73,bank: 0.00,yaw: 314.88 Emitters.Mode: 0
3,Player: t: 0.06,MSLAlt: 1279.81,speed(m/s): 236.72,heading: 5.50,x: -302916.37,y: 1279.81,z: 678096.65,pitch: 1.65,bank: 0.01,yaw: 314.88 Emitters.Mode: 0
4,Player: t: 0.11,MSLAlt: 1279.80,speed(m/s): 236.61,heading: 5.50,x: -302907.86,y: 1279.80,z: 678088.10,pitch: 1.49,bank: 0.03,yaw: 314.87 Emitters.Mode: 0
...,...,...,...,...,...,...,...,...,...,...
6262,Player: t: 79.24,MSLAlt: 1850.54,speed(m/s): 365.82,heading: 3.23,x: -313589.53,y: 1850.64,z: 669901.21,pitch: 3.81,bank: -0.23,yaw: 185.14 Emitters.1.Type.level3: 102 level...
6263,Player: t: 79.26,MSLAlt: 1850.90,speed(m/s): 365.82,heading: 3.23,x: -313595.47,y: 1850.94,z: 669900.62,pitch: 3.81,bank: -0.24,yaw: 185.14 Emitters.1.Type.level3: 102 level...
6264,Player: t: 79.27,MSLAlt: 1851.14,speed(m/s): 365.83,heading: 3.23,x: -313601.41,y: 1851.24,z: 669900.02,pitch: 3.81,bank: -0.24,yaw: 185.14 Emitters.1.Type.level3: 102 level...
6265,Player: t: 79.29,MSLAlt: 1851.50,speed(m/s): 365.83,heading: 3.23,x: -313607.74,y: 1851.56,z: 669899.39,pitch: 3.81,bank: -0.24,yaw: 185.14 Emitters.1.Type.level3: 102 level...


In [163]:
df.applymap(lambda x: re.sub(sub_pattern, '', x))

,Player:,MSLAlt:,speed(m/s):,heading:,x:,y:,z:,pitch:,bank:,yaw:
0,0.00,1279.81,240.83,5.50,-302927.78,1279.81,678108.13,1.74,-0.00,314.88 Emitters. 0
1,0.02,1279.81,240.83,5.50,-302924.88,1279.81,678105.21,1.74,0.00,314.88 Emitters. 0
2,0.04,1279.81,240.83,5.50,-302921.44,1279.81,678101.75,1.73,0.00,314.88 Emitters. 0
3,0.06,1279.81,236.72,5.50,-302916.37,1279.81,678096.65,1.65,0.01,314.88 Emitters. 0
4,0.11,1279.80,236.61,5.50,-302907.86,1279.80,678088.10,1.49,0.03,314.87 Emitters. 0
...,...,...,...,...,...,...,...,...,...,...
6262,79.24,1850.54,365.82,3.23,-313589.53,1850.64,669901.21,3.81,-0.23,185.14 Emitters.1.Type. -2. 0. 130. 0
6263,79.26,1850.90,365.82,3.23,-313595.47,1850.94,669900.62,3.81,-0.24,185.14 Emitters.1.Type. -2. 0. 130. 0
6264,79.27,1851.14,365.83,3.23,-313601.41,1851.24,669900.02,3.81,-0.24,185.14 Emitters.1.Type. -2. 0. 130. 0
6265,79.29,1851.50,365.83,3.23,-313607.74,1851.56,669899.39,3.81,-0.24,185.14 Emitters.1.Type. -2. 0. 130. 0


In [161]:
df

,Player:,MSLAlt:,speed(m/s):,heading:,x:,y:,z:,pitch:,bank:,yaw:
0,['Player: t: 0.00,MSLAlt: 1279.81,speed(m/s): 240.83,heading: 5.50,x: -302927.78,y: 1279.81,z: 678108.13,pitch: 1.74,bank: -0.00,yaw: 314.88 Emitters.Mode: 0
1,Player: t: 0.02,MSLAlt: 1279.81,speed(m/s): 240.83,heading: 5.50,x: -302924.88,y: 1279.81,z: 678105.21,pitch: 1.74,bank: 0.00,yaw: 314.88 Emitters.Mode: 0
2,Player: t: 0.04,MSLAlt: 1279.81,speed(m/s): 240.83,heading: 5.50,x: -302921.44,y: 1279.81,z: 678101.75,pitch: 1.73,bank: 0.00,yaw: 314.88 Emitters.Mode: 0
3,Player: t: 0.06,MSLAlt: 1279.81,speed(m/s): 236.72,heading: 5.50,x: -302916.37,y: 1279.81,z: 678096.65,pitch: 1.65,bank: 0.01,yaw: 314.88 Emitters.Mode: 0
4,Player: t: 0.11,MSLAlt: 1279.80,speed(m/s): 236.61,heading: 5.50,x: -302907.86,y: 1279.80,z: 678088.10,pitch: 1.49,bank: 0.03,yaw: 314.87 Emitters.Mode: 0
...,...,...,...,...,...,...,...,...,...,...
6262,Player: t: 79.24,MSLAlt: 1850.54,speed(m/s): 365.82,heading: 3.23,x: -313589.53,y: 1850.64,z: 669901.21,pitch: 3.81,bank: -0.23,yaw: 185.14 Emitters.1.Type.level3: 102 level...
6263,Player: t: 79.26,MSLAlt: 1850.90,speed(m/s): 365.82,heading: 3.23,x: -313595.47,y: 1850.94,z: 669900.62,pitch: 3.81,bank: -0.24,yaw: 185.14 Emitters.1.Type.level3: 102 level...
6264,Player: t: 79.27,MSLAlt: 1851.14,speed(m/s): 365.83,heading: 3.23,x: -313601.41,y: 1851.24,z: 669900.02,pitch: 3.81,bank: -0.24,yaw: 185.14 Emitters.1.Type.level3: 102 level...
6265,Player: t: 79.29,MSLAlt: 1851.50,speed(m/s): 365.83,heading: 3.23,x: -313607.74,y: 1851.56,z: 669899.39,pitch: 3.81,bank: -0.24,yaw: 185.14 Emitters.1.Type.level3: 102 level...
